Amadéo David (44761700) - Marine Van Renterghem (31621700) - Sylvie Van Schendel (44841700)

# LINMA1702 Optimisation d'un réseau de distribution d'eau

## I. Analyse d'un réseau existant

### I.1 Que représentent les expressions $A^Th$ et $Af$ ?

Soit h le vecteur contenant les hauteurs en chaque point, f le vecteur contenant les débits circulant dans les conduites et A la matrice d'incidence.

L'expression $A^Th$ est le vecteur contenant les dénivelés de chaque conduite.

L'expression $Af$ est le vecteur contenant les débits nets en chaque point.

### I.2 

### I.4 Résolution numérique du problème I.3 avec les données fournies

In [11]:
from scipy.optimize import linprog # import du solveur d'optimisation linéaire à utiliser
import matplotlib.pyplot as plt # pour graphiques éventuels
import numpy as np

In [12]:
#Fonction retournant [f,xC,xA] avec f la valeur de la fonction objectif maximale 
#xC un vecteur des valeurs optimales des débits aux points de consommation
#xA un vecteur des valeurs optimales des débits aux points d'approvisionnement
#Soit Pf le prix facturé
#Soit C le cout
#Soit DCmin et DCmax les valeurs minimales et maximales du débit aux points de consommation
#Soit DAmax les valeurs maximales du débit extractibles aux sources
def Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi,alpha):
    """
    Analyse prend comme argument, dans l'ordre :
        ° le cout;
        ° le prix facturé;
        ° la valeur minimale du débit aux points de consommation;
        ° la valeur maximale du débit aux points de consommation;
        ° la valeur maximale du débit aux points de productions.
    Analyse renvoie un vecteur retournant, dans l'ordre :
        ° la valeur de la fonction objectif maximale;
        ° un vecteur des valeurs optimales des débits aux points de consommation;
        ° un vecteur des valeurs optimales des débits aux points d'approvisionnement.
        
    """
    L=np.zeros(nConduites)
    for i in range(nConduites): 
        p=0
        P1=np.zeros(3)
        P2=np.zeros(3)
        while(A[p][i]==0): 
            p+=1
        P1[0]=X[p]
        P1[1]=Y[p]
        P1[2]=Z[p]
        for j in range(p+1,nPoints): 
            if A[j][i]==-1 or A[j][i]==1 : 
                P2[0]=X[j]
                P2[1]=Y[j]
                P2[2]=Z[j]
                L[i]+=np.sqrt((P2[0]-P1[0])**2+(P2[1]-P1[1])**2+(P2[2]-P1[2])**2)
                P1=P2.copy()
                P2=np.zeros(3)                
    h=A.T@Z
    print(A[np.array(Pc)-1])#Pourquoi ça fonctionne pas?
    print(A[np.array(Pa)-1])#Pourquoi ça fonctionne pas?
    A1=np.concatenate((A[np.array(Pc)-1],A[np.array(Pa)-1]))
    Debmax=np.multiply(np.outer(np.ones(len(A1)),alpha*R*R*h/L),A1)
    A2=A[np.array(Pc)-1]
    Debmin=-1*np.multiply(np.outer(np.ones(len(A2)),alpha*R*R*h/L),A2)
    Sum=np.sum(np.multiply(np.outer(np.ones(len(A1)),alpha*R*R*h/L),A1),axis=0)
    Aub=np.concatenate((np.concatenate((Debmax,Debmin)),Sum))
    c=np.multiply(np.multiply(np.outer(np.ones(len(A1)),alpha*R*R*h/L),A1).T,np.concatenate((Pf,C))).T
    #c  = -1* alpha*h*R**2/L * np.concatenate((Pf, -1*np.array(C)))       # On traite un problème de maximisation
    #A1 = np.eye(len(Dcmin)+len(Damax))
    #A2 = -1 * np.eye(len(Dcmin))                         # Matrice identite
    #A2Add = np.zeros((len(Dcmin),len(Damax)))            # Matrice de 0
    #A2 = np.concatenate((A2,A2Add),axis=1)               # Ajout de 0 a la fin de la matrice
    #A5  = np.concatenate((A1,A2))
    #A3 = np.ones((1,len(Dcmin)+len(Damax)))
    #A3[0,len(Dcmin):(len(Dcmin)+len(Damax))]*=-1
    #A6  = alpha*h*R**2/L*np.concatenate((A5,A3))
    b  = np.concatenate((np.concatenate((Dcmax,-1*np.array(Damax))),np.concatenate((-1*np.array(Dcmin),[0]))))
    res= linprog(c, A_ub = Aub, b_ub = b,bounds=(0, 1))#Modifier les bounds
    print(c)
    return [-1*res.fun, res.x[0:len(Dcmin)],res.x[len(Dcmin):len(Dcmin)+len(Damax)]]#modifier le return

In [1]:
def readFile(file_name):
    """
    Cette fonction lit un fichier formatté au type utilise dans les exemples du
    projet
    @pre                : Le fichier file_name est organise de la meme maniere
                          que les exemples donnes sur Moodle.
    
    @param file_name    : contient un String avec le nom du fichier a lire.
    
    @return n_points    : Contient un 'int' avec le nombre de points
    @return n_conduites : Contient un 'int' avec le nombre de conduites
    @return X           : Contient un 'numpy.array' avec les abscisses des points.
    @return Y           : Contient un 'numpy.array' avec les ordonnees des points.
    @return Z           : Contient un 'numpy.array' avec la hauteur des points.
    @return A           : Contient un 'numpy.array' avec la matrice d'incidence
                          des conduites.
    @return alpha       : Contient un 'float' avecla valeur de alpha.
    @return R           : Contient un 'float' avec le rayon des conduites.
    @return pa          : Contient un 'numpy.array' avec les points
                          d'approvisionnement.
    @return da_max      : Contient un 'numpy.array' avec les debits maximaux
                          extractibles.
    @return C           : Contient un 'numpy.array' avec les couts d'extraction.
    @return pc          : Contient un 'numpy.array' avec les points de
                          consommation.
    @return dc_min      : Contient un 'numpy.array' avec les debits minimaux
                          consommables.
    @return dc_max      : Contient un 'numpy.array' avec les debits maximaux
                          consommables.
    @return Pf          : Contient un 'float' avec le prix facture.
    @return pi          : Contient un 'numpy.array' avec les points
                          intermediaires.
    @return unit        : Contient un 'String' avec l'unite des coordonnes des
                          points.
    """
    with open(file_name,'r') as f:
        n_points    = (int)(f.readline().split()[0])
        n_conduites = (int)(f.readline().split()[0])
        f.readline()
        unit        = (f.readline().split(" = ")[1])
        
        coord       = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        X           = coord[:,0]
        Y           = coord[:,1]
        Z           = coord[:,2]
        
        f.readline()
        f.readline()
        A           = np.array(list(list(float(w) for w in f.readline().split()[:]) for i in range(n_points)))
        f.readline()
        
        UL_alpha    = f.readline().split(" = ")[1]
        UL_alpha    = UL_alpha.split()[0]
        UL2_alpha   = np.array(UL_alpha.split("^"))
        if(len(UL2_alpha) == 1):
            alpha   = (float)(UL2_alpha)
        else:
            alpha   = (float)(UL2_alpha[0]) ** (float)(UL2_alpha[1])
        
        R           = (float)(f.readline().split()[4])
        f.readline()
        
        UL_pa       = f.readline().split(" = [")[1]
        UL_pa       = UL_pa.split("]")[0]
        pa          = np.array(list(float(w) for w in UL_pa.split()[:]))
        
        UL_da_max   = f.readline().split(" = [")[1]
        UL_da_max   = UL_da_max.split("]")[0]
        da_max      = np.array(list(float(w) for w in UL_da_max.split()[:]))
        
        UL_C        = f.readline().split(" = [")[1]
        UL_C        = UL_C.split("]")[0]
        C           = np.array(list(float(w) for w in UL_C.split()[:]))
        
        f.readline()
        
        UL_pc       = f.readline().split(" = [")[1]
        UL_pc       = UL_pc.split("]")[0]
        pc          = np.array(list(float(w) for w in UL_pc.split()[:]))
        
        UL_dc_min   = f.readline().split(" = [")[1]
        UL_dc_min   = UL_dc_min.split("]")[0]
        dc_min      = np.array(list(float(w) for w in UL_dc_min.split()[:]))
        
        UL_dc_max   = f.readline().split(" = [")[1]
        UL_dc_max   = UL_dc_max.split("]")[0]
        dc_max      = np.array(list(float(w) for w in UL_dc_max.split()[:]))
        
        Pf          = (float)(f.readline().split()[3])
        
        f.readline()
        
        UL_pi       = f.readline().split("[")[1]
        UL_pi       = UL_pi.split("]")[0]
        pi          = np.array(list(float(w) for w in UL_pi.split()[:]))
        
    return (n_points, n_conduites, X, Y, Z, A, alpha, R, pa, da_max, C, pc, dc_min, dc_max, Pf, pi, unit)

In [20]:
(nPoints,nConduites,X,Y,Z,A, alpha,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf,Pi, unit)=readFile("Mini-exemple-avec-solution/Data-mini-exemple-avec-solution.txt")
print(Analyse(nPoints,nConduites,X,Y,Z,A,R,Pa,Damax,C,Pc,Dcmin,Dcmax,Pf*np.ones(len(Dcmin)),Pi,alpha))

[[ 0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  1.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  0.  0.
   0. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  1.  0.  0.  1.  1.
   0.  0.  0.]
 [ 0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  1. -1.]]
[[-1.  0.  0. -1. -1.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
   0.  0.  0.]
 [ 0.  0. -1.  0.  0.

ValueError: all the input arrays must have same number of dimensions

-0.0


# II. Améliorations du réseau

# III. Conception d'un réseau optimal